In [1]:
import pandas as pd
from selenium import webdriver
from lxml import etree
from time import sleep
import json

In [2]:
class Test_1_Spider():
    def __init__(self):
        #数据初始化
        self.basic_url = 'https://spa2.scrape.center/'
        #设置不自动关闭
        self.options = webdriver.ChromeOptions()
        self.options.add_experimental_option("detach", True)
        self.driver = webdriver.Chrome(executable_path="./chromedriver.exe",chrome_options=self.options)
    
    #构造url，翻页
    def get_url_list(self):
            url_list = [self.basic_url+f"page/{i}" for i in range(1,11)]
            # url_list = [self.basic_url]
            return url_list
    #获取响应
    def get_response(self,url):
        self.driver.get(url)
        self.driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        sleep(5)
        html_str = self.driver.page_source
        return html_str
    #清洗数据
    def get_data(self,html_str):
        HtmlData = etree.HTML(html_str)
        data = pd.DataFrame(columns=["电影名称","类型","产地","时长","得分"])
        div_list = HtmlData.xpath("//div[@class='el-card__body']")
        for div in div_list:
            info_dicts = {}
            info_dicts["电影名称"] = div.xpath(".//a[@class='name']/h2//text()")[0]
            info_dicts["类型"] = "-".join(div.xpath(".//div[@class='categories']//span/text()"))
            info_dicts["产地"] = div.xpath(".//div[@class='m-v-sm info'][1]/span[1]/text()")[0]
            info_dicts["时长"] = div.xpath(".//div[@class='m-v-sm info'][1]/span[3]/text()")[0]
            info_dicts["得分"] = div.xpath(".//p[@class='score m-t-md m-b-n-sm']/text()")[0].replace("\n                ","")
            info_df = pd.DataFrame(info_dicts,index=[0])
            data = pd.concat([data,info_df])

        return data



    def run(self):
         #运行主程序
         #1、获取url列表
        need_data = pd.DataFrame(columns=["电影名称","类型","产地","时长","得分"])
        url_list = self.get_url_list()
        for url in url_list:
            #2、发送请求，获取响应
            html_str = self.get_response(url)
            #3、清洗数据
            data = self.get_data(html_str)
            #保存数据
            need_data = pd.concat([need_data,data])
            sleep(2)
        need_data.to_excel(".\spider数据.xlsx",index=False)
        self.driver.quit()

        

In [3]:
if __name__ == "__main__":
    Spider = Test_1_Spider()
    Spider.run()

C:\Users\yuhuyi\AppData\Local\Temp\ipykernel_28148\2385686369.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(executable_path="./chromedriver.exe",chrome_options=self.options)
C:\Users\yuhuyi\AppData\Local\Temp\ipykernel_28148\2385686369.py:8: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(executable_path="./chromedriver.exe",chrome_options=self.options)
